<a href="https://colab.research.google.com/github/Tom-Jung/legendary-funicular/blob/main/Ch6_2.%20%EC%82%B0%EC%B6%9C%20%EB%B3%80%EB%8F%99%EC%9A%94%EC%9D%B8%20%EB%B6%84%ED%95%B4_3%EB%B6%80%EB%AC%B8_%ED%8C%8C%EC%89%90_%EB%9D%BC%EC%8A%A4%ED%8C%8C%EC%9D%B4%EB%A0%88%EC%8A%A4_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [649]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [650]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

### 산출 변동요인분해

In [651]:
# 2015년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2015_cst.csv"  # 불변 2015년 총거래표
Tot_cst_15 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2015_cst.csv"  # 불변 2015년 수입거래표
Imp_cst_15 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2015_cst.csv"  # 불변 2015년 국산거래표
Dom_cst_15 = pd.read_csv(url_dom)

In [652]:
# 2010년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Total_Matrix_2010_cst.csv"  # 불변 2010년 총거래표
Tot_cst_10 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Imported_Matrix_2010_cst.csv"  # 불변 2010년 수입거래표
Imp_cst_10 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/Domestic_Matrix_2010_cst.csv"  # 불변 2010년 국산거래표
Dom_cst_10 = pd.read_csv(url_dom)

In [653]:
Tot_cst_10.shape

(41, 47)

부문통합하여 3부문 IO표 만들기

In [654]:
cdd = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/code.csv"  # 코드매치 파일
ccdd = pd.read_csv(cdd)
cd_for_itr = ccdd.to_numpy().reshape(33,2)
cd_3 = cd_for_itr[:,1]
cd_33 = cd_for_itr[:,0]
print('3부문 코드 : \n', cd_3)
print('33부문 코드 : \n', cd_33)

3부문 코드 : 
 [1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
33부문 코드 : 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


In [655]:
# Mtx = np.zeros((3,32)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
# row=cd_3[:-2]
# col=cd_33[:-2]
# for i, j in zip(row,col):
#   Mtx[i-1,j-1] = 1
# print(Mtx)

In [656]:
code = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 3x33 매트릭스
       [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
code = code.astype(float)

In [657]:
SS_t0 = (Tot_cst_10.iloc[1:41,2:].to_numpy().astype(float)) # 10년 총거래표 numpy로 숫자만 불러오기(시간 t-1)
Im_t0 = (Imp_cst_10.iloc[1:38,2:].to_numpy().astype(float)) # 10년 수입거래표 numpy로 숫자만 불러오기

SS_t5 = (Tot_cst_15.iloc[1:41,2:].to_numpy().astype(float)) # 15년 총거래표 numpy로 숫자만 불러오기(시간 t)
Im_t5 = (Imp_cst_15.iloc[1:38,2:].to_numpy().astype(float)) # 15년 수입거래표 numpy로 숫자만 불러오기


In [658]:
nae_tot_row_t0 = code@SS_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_tot_row_t5 = code@SS_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [659]:
nae_tot_col_t0 = nae_tot_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_tot_col_t5 = nae_tot_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [660]:
nae_tot_col_t5

array([[3.225e+06, 3.906e+07, 1.190e+07],
       [1.775e+07, 9.254e+08, 3.509e+08],
       [7.137e+06, 2.617e+08, 5.753e+08]])

In [661]:
va_row_t0 = SS_t0[32:42,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(8X3)
va_row_t5 = SS_t5[32:41,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(7X3)
va_row_t5

array([[2.811e+07, 1.226e+09, 9.381e+08],
       [5.243e+06, 1.768e+08, 5.682e+08],
       [2.258e+07, 1.400e+08, 2.789e+08],
       [4.828e+06, 1.148e+08, 1.900e+08],
       [6.100e+05, 5.344e+07, 8.205e+07],
       [3.326e+07, 4.850e+08, 1.119e+09],
       [6.137e+07, 1.711e+09, 2.057e+09]])

In [662]:
fd_cd = np.array([[1,1,0,0,0,0,0,0,0],   # 최종수요를 통합하기 위한 0과 1로된 행렬
                  [0,0,1,1,1,1,0,0,0],
                  [0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,1]])


In [663]:
fd_t0 = SS_t0[0:32, 33:42]
fd_t5 = SS_t5[0:32, 33:42]

In [664]:
fd_row_t0=code@fd_t0@fd_cd.T
fd_row_t5=code@fd_t5@fd_cd.T
fd_row_t0

array([[1.601e+07, 2.646e+06, 8.298e+05, 1.949e+07, 6.890e+07],
       [1.665e+08, 1.374e+08, 4.565e+08, 7.604e+08, 1.848e+09],
       [7.403e+08, 3.104e+08, 7.855e+07, 1.129e+09, 1.819e+09]])

In [665]:
fd_row_sm_t0 = np.sum((fd_row_t0), axis=0) # 최종수요의 행합
fd_row_sm_t5 = np.sum((fd_row_t5), axis=0) # 최종수요의 행합

In [666]:
fd_sm_t0=fd_row_sm_t0.reshape(1,5)
fd_sm_t5=fd_row_sm_t5.reshape(1,5)

In [667]:
nae_va_ct_t0 = np.concatenate((nae_tot_col_t0, va_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_va_ct_t5 = np.concatenate((nae_tot_col_t5, va_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [668]:
nae_va_ct_sm_t0 = nae_va_ct_t0.sum(axis=1).reshape(11,1)  # 부가가치 및 산출액의 열합
nae_va_ct_sm_t5 = nae_va_ct_t5.sum(axis=1).reshape(10,1)  # 부가가치 및 산출액의 열합

In [669]:
nnull_t0 = np.zeros((7,5)) # 0행렬
nnull_t5 = np.zeros((6,5)) # 0행렬

In [670]:
fd_nnull_t0 = np.concatenate((fd_row_t0, fd_sm_t0, nnull_t0),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기
fd_nnull_t5 = np.concatenate((fd_row_t5, fd_sm_t5, nnull_t5),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기

In [671]:
tot_f_t0 = np.concatenate((nae_va_ct_t0,nae_va_ct_sm_t0, fd_nnull_t0), axis=1)  # 최종 3x3 부문통합된 거래표
tot_f_t5 = np.concatenate((nae_va_ct_t5,nae_va_ct_sm_t5, fd_nnull_t5), axis=1)  # 최종 3x3 부문통합된 거래표

In [672]:
tot_F_t0 = pd.DataFrame(tot_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','조정항', '부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [673]:
tot_F_t5 = pd.DataFrame(tot_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [674]:
# 2010, 2015년 3부문 불변표 내보내기
tot_F_t0.to_csv('10년_총거래표_3부문_불변.csv', encoding="utf-8-sig")
tot_F_t5.to_csv('15년_총거래표_3부문_불변.csv', encoding="utf-8-sig")

### 수입거래표 3부문 통합

In [675]:
nae_imp_row_t0 = code@Im_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_imp_row_t5 = code@Im_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [676]:
nae_imp_col_t0 = nae_imp_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_imp_col_t5 = nae_imp_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [677]:
Im_t0.shape

(33, 42)

In [678]:
ism_row_t0 = (Im_t0[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5 = (Im_t5[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5

array([[1.881e+06, 3.520e+08, 1.233e+08]])

In [679]:
ifd_t0 = Im_t0[0:32, 33:42] #수입최종수요
ifd_t5 = Im_t5[0:32, 33:42] #수입최종수요

In [680]:
ifd_row_t0=code@ifd_t0@fd_cd.T
ifd_row_t5=code@ifd_t5@fd_cd.T

In [681]:
ifd_row_sm_t0 = np.sum((ifd_row_t0), axis=0) # 최종수요의 행합
ifd_row_sm_t5 = np.sum((ifd_row_t5), axis=0) # 최종수요의 행합

In [682]:
ifd_sm_t0=ifd_row_sm_t0.reshape(1,5)
ifd_sm_t5=ifd_row_sm_t5.reshape(1,5)

In [683]:
ism_row_t0.ndim

2

In [684]:
nae_im_t0 = np.concatenate((nae_imp_col_t0, ism_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_im_t5 = np.concatenate((nae_imp_col_t5, ism_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [685]:
nae_col_sm_imp_t0 = nae_im_t0.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합
nae_col_sm_imp_t5 = nae_im_t5.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합

In [686]:
nae_im_sm_t0 = nae_col_sm_imp_t0.sum(axis=1).reshape(4,1)  # 중간투입 및 중간투입 열합
nae_im_sm_t5 = nae_col_sm_imp_t5.sum(axis=1).reshape(4,1)

In [687]:
ifd_nnull_t0 = np.concatenate((ifd_row_t0, ifd_sm_t0),axis=0)  #   최종수요 및 최종수요 행합 합치기
ifd_nnull_t5 = np.concatenate((ifd_row_t5, ifd_sm_t5),axis=0)  #   최종수요 및 최종수요 행합 합치기

In [688]:
imp_f_t0 = (np.concatenate((nae_im_t0, nae_im_sm_t0, ifd_nnull_t0), axis=1))  # 최종 3x3 부문통합된 거래표
imp_f_t5 = (np.concatenate((nae_im_t5, nae_im_sm_t5, ifd_nnull_t5), axis=1))  # 최종 3x3 부문통합된 거래표

In [689]:
imp_F_t0 = pd.DataFrame(imp_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5 = pd.DataFrame(imp_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5

,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
농산품,348818.0,8218892.0,748604.0,9316314.0,2859389.0,-17714.0,0.0,2841675.0,12157989.0
공산품,1157164.0,327866649.0,76354626.0,405378439.0,53140499.0,53398716.0,2125896.0,108665111.0,514043550.0
서비스,374663.0,15940859.0,46232222.0,62547744.0,28500448.0,6752632.0,0.0,35253080.0,97800824.0
중간투입계,1880645.0,352026400.0,123335452.0,477242497.0,84500336.0,60133634.0,2125896.0,146759866.0,624002363.0


In [690]:
# 2010, 2015년 3부문 불변표 내보내기
imp_F_t0.to_csv('10년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")
imp_F_t5.to_csv('15년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")

In [691]:
S1 = tot_f_t5.astype(float)
I1 = imp_f_t5.astype(float)
S0 = tot_f_t0.astype(float)
I0 = imp_f_t0.astype(float)
S1.sum()

40931720084.0

In [692]:
SS1 = np.concatenate((S1[:8,],np.zeros((1,9)), S1[8:10,] ), axis=0)  # 2015년 부가가치 조정항을 0으로 삽입(2010년과 동일한 포맷을 유지하기 위함)

In [693]:
SS0 = S0[0:3,0:3].astype(float)  # 내생항목 3x3
SS1 = S1[0:3,0:3].astype(float)
SS1.shape

(3, 3)

In [694]:
II0 = I0[0:3,0:3]  # 수입 내생항목 3x3
II1 = I1[0:3,0:3]

In [695]:
DD0 =SS0 - II0  # 국산 내생항목 3x3
DD1 =SS1 - II1
DD0

array([[4.129e+06, 2.900e+07, 8.603e+06],
       [1.676e+07, 5.103e+08, 2.352e+08],
       [5.354e+06, 2.091e+08, 4.273e+08]])

In [696]:
TM0 = I0[0:3,8]  # 총수입벡터
TM1 = I1[0:3,8]
TM0

array([9.798e+06, 4.169e+08, 7.243e+07])

In [697]:
IM0 = I0[0:3,3]  # 중간재수입벡터
IM1 = I1[0:3,3]
IM0

array([7.690e+06, 3.251e+08, 4.833e+07])

In [698]:
IF0 = I0[0:3,7]  # 최종재수입벡터
IF1 = I1[0:3,7]
IF0

array([ 2108171., 91865740., 24096241.])

In [699]:
SFC0 = S0[0:3,4]  # 최종수요-소비
SFC1 = S1[0:3,4]
SFC0

array([1.601e+07, 1.665e+08, 7.403e+08])

In [700]:
SFI0 = S0[0:3,5] # 최종수요-투자
SFI1 = S1[0:3,5]
SFI0/1000000

array([  2.646, 137.44 , 310.398])

In [701]:
SF0 = S0[0:3,7] # 최종수요합계
SF1 = S1[0:3,7]
SF0/1000000

array([  19.485,  760.437, 1129.235])

In [702]:
EX0 = S0[0:3,6] # 수출
EX1 = S1[0:3,6]
EX0/1000000

array([  0.83 , 456.483,  78.546])

In [703]:
DF0 = SF0 - EX0 #소비+투자
DF1 = SF1 - EX1
DF0/1000000

array([  18.656,  303.955, 1050.689])

In [704]:
SFC00 = SFC0.sum(axis=0) # 소비합계
SFC11 = SFC1.sum(axis=0)

SFI00 = SFI0.sum(axis=0) # 투자합계
SFI11 = SFI1.sum(axis=0)

EX00 = EX0.sum(axis=0) # 수출합계
EX11 = EX1.sum(axis=0)

print('2010년 소비합계 : ', SFC00)
print('2015년 소비합계 : ', SFC11)

print('2010년 투자합계 : ', SFI00)
print('2015년 투자합계 : ', SFI11)

print('2010년 수출합계 : ', EX00)
print('2015년 수출합계 : ', EX11)


2010년 소비합계 :  922814206.0
2015년 소비합계 :  1054900320.0
2010년 투자합계 :  450484823.0
2015년 투자합계 :  489601534.0
2010년 수출합계 :  535858572.0
2015년 수출합계 :  716951177.0


In [705]:
GDC1 = ((SFC11 - SFC00)/ SFC00) # 소비증가율
GDC2 = GDC1*SFC0  # 규모효과
GDC3 = (SFC1-SFC0) - GDC2 # 구조효과

In [706]:
GDI1 = ((SFI11 - SFI00)/ SFI00) # 투자증가율
GDI2 = GDI1*SFI0 # 규모효과
GDI3 = (SFI1-SFI0) - GDI2 # 구조효과

In [707]:
ED1 = ((EX11 - EX00)/EX00) # 수출증가율
ED2 = ED1*EX0 # 규모효과
ED3 = (EX1-EX0) - ED2 # 구조효과

In [708]:
C0 = S0[-1,0:3].T.reshape([3,1])  # 총산출액
C1 = S1[-1,0:3].T.reshape([3,1])
C0/1000000

array([[  59.105],
       [1430.838],
       [1746.898]])

In [709]:
TC0 = C0.sum(axis=0)  #총산출계
TC1 = C1.sum(axis=0)
TC0/1000000

array([3236.841])

In [710]:
V0 = S0[-1,0:3].T - S0[3,0:3].T # 부가가치계벡터 = 총투입액 - 중간투입계
V1 = S1[-1,0:3].T - S1[3,0:3].T
V1/1000000

array([  33.258,  485.04 , 1119.152])

In [711]:
AV0 = V0/C0 # 부가가치계수
AV1 = V1/C1
AV1

array([[ 0.542,  7.903, 18.236],
       [ 0.019,  0.283,  0.654],
       [ 0.016,  0.236,  0.544]])

In [712]:
C0

array([[5.911e+07],
       [1.431e+09],
       [1.747e+09]])

In [713]:
CT0 = np.tile(C0, reps=[1,3])   # Ct벡터를 반복해여 행렬(3x3)로 만듬
CT1 = np.tile(C1, reps=[1,3])
CT1

array([[6.137e+07, 6.137e+07, 6.137e+07],
       [1.711e+09, 1.711e+09, 1.711e+09],
       [2.057e+09, 2.057e+09, 2.057e+09]])

In [714]:
AS0 = SS0/CT0.T # 총투입계수표
AS1 = SS1/CT1.T
# np.isnan(AS_t)

In [715]:
AD0 = DD0/CT0.T # 국산투입계수표
AD1 = DD1/CT1.T

In [716]:
II0 = AS0-AD0 # 수입투입계수표
II1 = AS1-AD1

In [717]:
M0 = IF0/DF0 # 수입최종수요/(소비+투자)
M0 = np.diag(M0)

M1 = IF1/DF1
M1 = np.diag(M1)

In [718]:
RD0 = np.linalg.inv(np.eye(3) - AD0) # 생산유발계수
RD1 = np.linalg.inv(np.eye(3) - AD1)

In [719]:
AM0 = II0/AS0 #조정항목
AM0 = AM0*AS1

AM1 = II1/AS1
AM1 = AM1*AS0

# np.nan_to_num(AM0, copy=False)
# np.nan_to_num(AM1, copy=False)

In [720]:
AA = AS1 - AS0 #투입계수변화
AS1_f = pd.DataFrame(AS1)
AS1_f.to_csv('AS1_f.csv')

#### 산출액성장요인분해
-  파쉐 및 라스파이레스 방식으로 산출액 변동을 분해하여 평균으로 성장요인을 분해
- 파쉐 방식

In [721]:
# TC1 = ((RD1@(np.eye(3)-M1))@GDC2).reshape([3,1])  # 소비규모증가효과
# TC11 = ((RD1@(np.eye(3)-M1))@GDC3).reshape([3,1]) # 소비구조증가효과
# TCTD = TC1 + TC11
# TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# # TC111

In [722]:
# TI1 = ((RD1@(np.eye(3)-M1))@GDI2).reshape([3,1])  # 소비규모증가효과
# TI11 = ((RD1@(np.eye(3)-M1))@GDI3).reshape([3,1]) # 소비구조증가효과
# TITD = TI1 + TI11
# TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [723]:
# TQ = TC1+TI1 # 국내최종수요규모증가효과
# TS = TC11+TI11 # 국내최종수요규조변화효과
# TQS = TQ+TS
# T1 = np.concatenate((TQS, TQ, TS), axis=1)
# T1.shape
# TQS/1000000

In [724]:
TQS=(RD1@(np.eye(3)-M1))@(DF1-DF0).reshape([3,1])
# TQS/1000000

In [725]:
TTE= RD1@(EX1-EX0).reshape([3,1]) # 수출효과
# TTE/1000000

In [726]:
T3 = ((RD1@(M1 - M0))@DF0).reshape([3,1]) # 최종재수입대체효과
T3 = T3*-1
T4 = ((RD1@(AM1 - II0)))@C0[0:3,] # 중간재수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
T4

array([[ -1444844.583],
       [-24665224.464],
       [ -8593842.803]])

In [727]:
T5 = (RD1@(AA-(II1-AM1)))@C0 # 기술변화효과
T5.shape
T5

array([[-2655781.777],
       [ 6683402.413],
       [30749629.274]])

In [728]:
DCT0 = C1-C0  #산출액 실질변동액
DCT1 = TQS + TTE + T3 + T4 + T5 # 총변동요인
DCT = DCT0-DCT1

In [729]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
PGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
# PGD1015_f = (PGD1015/1000).round(1)
# PGD1015_f

In [730]:
PGD1015_ff = pd.DataFrame((PGD1015_f/1000000).round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [731]:
PGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,2.266,2.266,0.0,2.393,5.203,-1.228,-1.445,-2.656
공산품,280.315,280.315,0.0,61.723,256.975,-20.400,-24.665,6.683
서비스,310.315,310.315,0.0,214.444,86.770,-13.055,-8.594,30.750
계,592.897,592.897,0.0,278.560,348.947,-34.684,-34.704,34.777


- 라스파이레스식

In [732]:
# TC1 = ((RD0@(np.eye(3)-M0))@GDC2).reshape([3,1])  # 소비규모증가효과
# TC11 = ((RD0@(np.eye(3)-M0))@GDC3).reshape([3,1]) # 소비구조증가효과
# TCTD = TC1 + TC11
# TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# # TC111

In [733]:
# TI1 = ((RD0@(np.eye(3)-M0))@GDI2).reshape([3,1])  # 소비규모증가효과
# TI11 = ((RD0@(np.eye(3)-M0))@GDI3).reshape([3,1]) # 소비구조증가효과
# TITD = TI1 + TI11
# TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [734]:
# TQ = TC1+TI1 # 국내최종수요규모증가효과
# TS = TC11+TI11 # 국내최종수요규조변화효과
# TQS = TQ+TS
# T1 = np.concatenate((TQS, TQ, TS), axis=1)
# T1.shape

In [735]:
TQS=(RD0@(np.eye(3)-M0))@(DF1-DF0).reshape([3,1])
# TQS/1000000

In [736]:
T2 = (RD0@ED2).reshape([3,1]) # 수출규모증가효과
T22 = (RD0@ED3) .reshape([3,1])# 수출구조변화효과
TTE = T2+T22
T2 = np.concatenate((TTE, T2, T22), axis=1)

In [737]:
TTE= RD0@(EX1-EX0).reshape([3,1]) # 수출효과
# TTE/1000000

In [738]:
T3 = ((RD0@(M1 - M0))@DF1).reshape([3,1]) # 최종재수입대체효과
T3 = T3*-1
T4 = ((RD0@(II1 - AM0)))@C1[0:3,] # 중간재수입대체효과
T4 = T4*-1
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
TI

array([[ -3052300.402,  -1343348.115,  -1708952.286],
       [-51759853.002, -21982022.725, -29777830.277],
       [-24702163.762, -14486876.999, -10215286.763]])

In [739]:
T5 = (RD0@(AA-(AM0-II0)))@C1 # 기술변화효과
T5.shape
T5

array([[-3181775.34 ],
       [ 7977605.378],
       [35497375.094]])

In [740]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
LGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
# LGD1015_f = (LGD1015/1000).round(1)
LGD1015_f

array([[ 2.266e+06,  2.266e+06,  2.076e-01,  2.517e+06,  5.983e+06,
        -1.343e+06, -1.709e+06, -3.182e+06],
       [ 2.803e+08,  2.803e+08,  9.984e+00,  6.352e+07,  2.606e+08,
        -2.198e+07, -2.978e+07,  7.978e+06],
       [ 3.103e+08,  3.103e+08,  3.307e+00,  2.127e+08,  8.684e+07,
        -1.449e+07, -1.022e+07,  3.550e+07],
       [ 5.929e+08,  5.929e+08,  1.350e+01,  2.787e+08,  3.534e+08,
        -3.781e+07, -4.170e+07,  4.029e+07]])

In [741]:
LGD1015_ff = pd.DataFrame((LGD1015_f/1000000).round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [742]:
LGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,2.266,2.266,0.0,2.517,5.983,-1.343,-1.709,-3.182
공산품,280.315,280.315,0.0,63.521,260.576,-21.982,-29.778,7.978
서비스,310.315,310.315,0.0,212.676,86.844,-14.487,-10.215,35.497
계,592.897,592.897,0.0,278.714,353.403,-37.812,-41.702,40.293


In [743]:
PGD1015_ff.to_csv('산출액변동요인_파쉐.csv', encoding="utf-8-sig")
LGD1015_ff.to_csv('산출액변동요인_라스파이레스.csv', encoding="utf-8-sig")

In [744]:
Ctbr_Pa =  PGD1015_f[:,3:]
Ctbr_La =  LGD1015_f[:,3:]
Ctbr_La

array([[ 2.517e+06,  5.983e+06, -1.343e+06, -1.709e+06, -3.182e+06],
       [ 6.352e+07,  2.606e+08, -2.198e+07, -2.978e+07,  7.978e+06],
       [ 2.127e+08,  8.684e+07, -1.449e+07, -1.022e+07,  3.550e+07],
       [ 2.787e+08,  3.534e+08, -3.781e+07, -4.170e+07,  4.029e+07]])

In [745]:
sm_gap_Pa = PGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap_La = LGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap_La

array([[2.266e+06],
       [2.803e+08],
       [3.103e+08],
       [5.929e+08]])

In [746]:
sm_gap1_Pa = np.tile(sm_gap_Pa, reps=[1,5]) # 요인별 합계를 33x16으로 반복
sm_gap1_La = np.tile(sm_gap_La, reps=[1,5]) # 요인별 합계를 33x16으로 반복
sm_gap1_La

array([[2.266e+06, 2.266e+06, 2.266e+06, 2.266e+06, 2.266e+06],
       [2.803e+08, 2.803e+08, 2.803e+08, 2.803e+08, 2.803e+08],
       [3.103e+08, 3.103e+08, 3.103e+08, 3.103e+08, 3.103e+08],
       [5.929e+08, 5.929e+08, 5.929e+08, 5.929e+08, 5.929e+08]])

In [747]:
Ctrb_f_Pa = (Ctbr_Pa/sm_gap1_Pa)*100  # 각 항목별 기여율
Ctrb_f_La = (Ctbr_La/sm_gap1_La)*100  # 각 항목별 기여율
Ctrb_f_Pa

array([[ 105.579,  229.562,  -54.202,  -63.753, -117.186],
       [  22.019,   91.673,   -7.278,   -8.799,    2.384],
       [  69.105,   27.962,   -4.207,   -2.769,    9.909],
       [  46.983,   58.855,   -5.85 ,   -5.853,    5.866]])

In [748]:
Ctrb_ff_Pa = pd.DataFrame(Ctrb_f_Pa,columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [749]:
Ctrb_ff_La = pd.DataFrame(Ctrb_f_La,columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [750]:
Ctrb_ff_Pa.to_csv('항목별 기여율_파쉐.csv', encoding="utf-8-sig")
Ctrb_ff_La.to_csv('항목별 기여율_라스파이레스.csv', encoding="utf-8-sig")